# 1 - Introduction

This notebook train four different neural network models. They are all pretty similar.

## 1.1 Load Packages and Global Variables

In [1]:
%matplotlib inline
import os
import numpy as np
import nltk
from sklearn.externals import joblib
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [2]:
PROJECT_DIR = os.path.join(os.getcwd(), os.pardir)
os.chdir(PROJECT_DIR)

In [3]:
import src.neural_networks as nn

In [4]:
from importlib import reload
nn = reload(nn)

## 1.2- Load the Data

Load the train and validation neural network data sets if they are present, otherwise raise an exception.

In [5]:
#Load sequnces
try:
    train = joblib.load('data/processed/train_nn.pkl')
    valid = joblib.load('data/processed/valid_nn.pkl')
except FileNotFoundError:
    #need to run earlier notebook if files not present
    raise Exception("Files not found. Run Notebook 4.1")

In [6]:
#Load embedding matrix of 50 dimensions
try:
    embedding_matrix50 = joblib.load('data/interim/embeddings50.pkl')
except FileNotFoundError:
    #need to run earlier notebook if files not present
    raise Exception("Files not found. Run Notebook 4.1")

In [7]:
#Load embedding matrix of 300 dimensions
try:
    embedding_matrix300 = joblib.load('data/interim/embeddings300.pkl')
except FileNotFoundError:
    #need to run earlier notebook if files not present
    raise Exception("Files not found. Run Notebook 4.1")

In [8]:
#these arguments will be the same when training both models
run_args = {"train":(train["seqs"],train["labels"]),
            "valid":(valid["seqs"],valid["labels"])}

# 2 - Train 50d model with fixed embeddings

In [9]:
#LSTM, average final
model50_fixed = nn.create_model(embedding_matrix=embedding_matrix50, n_hidden=150, train_embed=False)
model50_fixed.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
joke_seq (InputLayer)        (None, 300)               0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 50)           1369800   
_________________________________________________________________
mask_paddings (Masking)      (None, 300, 50)           0         
_________________________________________________________________
drop_words (SpatialDropout1D (None, 300, 50)           0         
_________________________________________________________________
mask_dropped_words (Masking) (None, 300, 50)           0         
_________________________________________________________________
reccurrent_layer (LSTM)      (None, 150)               120600    
_________________________________________________________________
drop_dense (Dropout)         (None, 150)               0         
__________

In [10]:
#output model as svg plot
plot_model(model50_fixed, to_file='reports/figures/nn_50d.svg', show_shapes =True)

In [11]:
%%time
#now train
history50_fixed = nn.run_model(model=model50_fixed, out_path="models/nn_50d_fixed.hdf5", **run_args)

Train on 171945 samples, validate on 24564 samples
Epoch 1/1000
171945/171945 [==============================] - 136s 789us/step - loss: 0.6713 - acc: 0.5802 - val_loss: 0.6625 - val_acc: 0.5970
Epoch 2/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6613 - acc: 0.5963 - val_loss: 0.6578 - val_acc: 0.6029
Epoch 3/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6601 - acc: 0.5956 - val_loss: 0.6549 - val_acc: 0.6074
Epoch 4/1000
171945/171945 [==============================] - 90s 526us/step - loss: 0.6562 - acc: 0.6027 - val_loss: 0.6493 - val_acc: 0.6087
Epoch 5/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6548 - acc: 0.6038 - val_loss: 0.6485 - val_acc: 0.6109
Epoch 6/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6519 - acc: 0.6077 - val_loss: 0.6466 - val_acc: 0.6163
Epoch 7/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.650

Epoch 27/1000
171945/171945 [==============================] - 91s 530us/step - loss: 0.6379 - acc: 0.6298 - val_loss: 0.6301 - val_acc: 0.6390
Epoch 28/1000
171945/171945 [==============================] - 91s 527us/step - loss: 0.6364 - acc: 0.6309 - val_loss: 0.6315 - val_acc: 0.6409
Epoch 29/1000
171945/171945 [==============================] - 90s 526us/step - loss: 0.6363 - acc: 0.6324 - val_loss: 0.6295 - val_acc: 0.6401
Epoch 30/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6354 - acc: 0.6341 - val_loss: 0.6301 - val_acc: 0.6413
Epoch 31/1000
171945/171945 [==============================] - 90s 526us/step - loss: 0.6350 - acc: 0.6343 - val_loss: 0.6284 - val_acc: 0.6429
Epoch 32/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6350 - acc: 0.6348 - val_loss: 0.6278 - val_acc: 0.6422
Epoch 33/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6338 - acc: 0.6358 - val_loss: 0.6281 - val_acc:

171945/171945 [==============================] - 92s 532us/step - loss: 0.6249 - acc: 0.6475 - val_loss: 0.6190 - val_acc: 0.6571
Epoch 55/1000
171945/171945 [==============================] - 90s 526us/step - loss: 0.6245 - acc: 0.6493 - val_loss: 0.6193 - val_acc: 0.6573
Epoch 56/1000
171945/171945 [==============================] - 90s 526us/step - loss: 0.6243 - acc: 0.6488 - val_loss: 0.6203 - val_acc: 0.6537
Epoch 57/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6232 - acc: 0.6497 - val_loss: 0.6191 - val_acc: 0.6553
Epoch 58/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6235 - acc: 0.6483 - val_loss: 0.6177 - val_acc: 0.6593
Epoch 59/1000
171945/171945 [==============================] - 90s 526us/step - loss: 0.6223 - acc: 0.6510 - val_loss: 0.6192 - val_acc: 0.6555
Epoch 60/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6225 - acc: 0.6511 - val_loss: 0.6213 - val_acc: 0.6552
Epoch 

Epoch 83/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6150 - acc: 0.6599 - val_loss: 0.6153 - val_acc: 0.6608
Epoch 84/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6154 - acc: 0.6596 - val_loss: 0.6138 - val_acc: 0.6599
Epoch 85/1000
171945/171945 [==============================] - 90s 523us/step - loss: 0.6144 - acc: 0.6605 - val_loss: 0.6163 - val_acc: 0.6597
Epoch 86/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6143 - acc: 0.6610 - val_loss: 0.6141 - val_acc: 0.6619
Epoch 87/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6146 - acc: 0.6584 - val_loss: 0.6138 - val_acc: 0.6624
Epoch 88/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6141 - acc: 0.6594 - val_loss: 0.6140 - val_acc: 0.6604
Epoch 89/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6135 - acc: 0.6607 - val_loss: 0.6120 - val_acc:

Epoch 113/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6089 - acc: 0.6664 - val_loss: 0.6115 - val_acc: 0.6637
Epoch 114/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6092 - acc: 0.6651 - val_loss: 0.6103 - val_acc: 0.6648
Epoch 115/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6088 - acc: 0.6667 - val_loss: 0.6124 - val_acc: 0.6617
Epoch 116/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6090 - acc: 0.6659 - val_loss: 0.6093 - val_acc: 0.6657
Epoch 117/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6086 - acc: 0.6667 - val_loss: 0.6099 - val_acc: 0.6658
Epoch 118/1000
171945/171945 [==============================] - 90s 526us/step - loss: 0.6085 - acc: 0.6671 - val_loss: 0.6102 - val_acc: 0.6658
Epoch 119/1000
171945/171945 [==============================] - 90s 526us/step - loss: 0.6077 - acc: 0.6681 - val_loss: 0.6088 - v

Epoch 143/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6051 - acc: 0.6702 - val_loss: 0.6084 - val_acc: 0.6670
Epoch 144/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6049 - acc: 0.6714 - val_loss: 0.6083 - val_acc: 0.6680
Epoch 145/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6047 - acc: 0.6714 - val_loss: 0.6111 - val_acc: 0.6638
Epoch 146/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6042 - acc: 0.6717 - val_loss: 0.6088 - val_acc: 0.6674
Epoch 147/1000
171945/171945 [==============================] - 90s 526us/step - loss: 0.6039 - acc: 0.6707 - val_loss: 0.6097 - val_acc: 0.6641
Epoch 148/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6046 - acc: 0.6703 - val_loss: 0.6101 - val_acc: 0.6646
Epoch 149/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6045 - acc: 0.6709 - val_loss: 0.6090 - v

# 3 - Train 300d model with fixed embeddings

Second model 300 dimensionhal word embeddings. However the number of hidden units is decreases in order mtianta  aprocaml the same number of trainable paramters.

In [12]:
#LSTM, average final
model300_fixed = nn.create_model(embedding_matrix=embedding_matrix300, n_hidden=150)
model300_fixed.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
joke_seq (InputLayer)        (None, 300)               0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 300)          8218800   
_________________________________________________________________
mask_paddings (Masking)      (None, 300, 300)          0         
_________________________________________________________________
drop_words (SpatialDropout1D (None, 300, 300)          0         
_________________________________________________________________
mask_dropped_words (Masking) (None, 300, 300)          0         
_________________________________________________________________
reccurrent_layer (LSTM)      (None, 150)               270600    
_________________________________________________________________
drop_dense (Dropout)         (None, 150)               0         
__________

In [13]:
%%time
#now train
history300_fixed = nn.run_model(model=model300_fixed, out_path="models/nn_300d_fixed.hdf5", **run_args)

Train on 171945 samples, validate on 24564 samples
Epoch 1/1000
171945/171945 [==============================] - 143s 834us/step - loss: 0.6592 - acc: 0.6016 - val_loss: 0.6476 - val_acc: 0.6214
Epoch 2/1000
171945/171945 [==============================] - 97s 564us/step - loss: 0.6501 - acc: 0.6154 - val_loss: 0.6419 - val_acc: 0.6242
Epoch 3/1000
171945/171945 [==============================] - 97s 565us/step - loss: 0.6431 - acc: 0.6245 - val_loss: 0.6360 - val_acc: 0.6354
Epoch 4/1000
171945/171945 [==============================] - 97s 566us/step - loss: 0.6396 - acc: 0.6279 - val_loss: 0.6342 - val_acc: 0.6360
Epoch 5/1000
171945/171945 [==============================] - 97s 565us/step - loss: 0.6374 - acc: 0.6312 - val_loss: 0.6326 - val_acc: 0.6383
Epoch 6/1000
171945/171945 [==============================] - 97s 564us/step - loss: 0.6341 - acc: 0.6366 - val_loss: 0.6302 - val_acc: 0.6403
Epoch 7/1000
171945/171945 [==============================] - 97s 565us/step - loss: 0.631

Epoch 27/1000
171945/171945 [==============================] - 97s 564us/step - loss: 0.5938 - acc: 0.6812 - val_loss: 0.6095 - val_acc: 0.6679
Epoch 28/1000
171945/171945 [==============================] - 97s 564us/step - loss: 0.5922 - acc: 0.6845 - val_loss: 0.6121 - val_acc: 0.6663
Epoch 29/1000
171945/171945 [==============================] - 97s 565us/step - loss: 0.5899 - acc: 0.6861 - val_loss: 0.6065 - val_acc: 0.6707
Epoch 30/1000
171945/171945 [==============================] - 97s 565us/step - loss: 0.5882 - acc: 0.6870 - val_loss: 0.6074 - val_acc: 0.6712
Epoch 31/1000
171945/171945 [==============================] - 97s 565us/step - loss: 0.5870 - acc: 0.6889 - val_loss: 0.6077 - val_acc: 0.6692
Epoch 32/1000
171945/171945 [==============================] - 97s 564us/step - loss: 0.5862 - acc: 0.6886 - val_loss: 0.6069 - val_acc: 0.6723
Epoch 33/1000
171945/171945 [==============================] - 97s 564us/step - loss: 0.5854 - acc: 0.6892 - val_loss: 0.6077 - val_acc:

Epoch 57/1000
171945/171945 [==============================] - 97s 565us/step - loss: 0.5609 - acc: 0.7110 - val_loss: 0.6075 - val_acc: 0.6760
Epoch 58/1000
171945/171945 [==============================] - 97s 564us/step - loss: 0.5598 - acc: 0.7129 - val_loss: 0.6053 - val_acc: 0.6768
Epoch 59/1000
171945/171945 [==============================] - 97s 564us/step - loss: 0.5582 - acc: 0.7123 - val_loss: 0.6069 - val_acc: 0.6764
Epoch 60/1000
171945/171945 [==============================] - 97s 564us/step - loss: 0.5579 - acc: 0.7140 - val_loss: 0.6047 - val_acc: 0.6785
Epoch 61/1000
171945/171945 [==============================] - 97s 564us/step - loss: 0.5579 - acc: 0.7141 - val_loss: 0.6039 - val_acc: 0.6780
Epoch 62/1000
171945/171945 [==============================] - 97s 564us/step - loss: 0.5564 - acc: 0.7153 - val_loss: 0.6091 - val_acc: 0.6759
Epoch 63/1000
171945/171945 [==============================] - 97s 564us/step - loss: 0.5549 - acc: 0.7169 - val_loss: 0.6064 - val_acc:

# 4 - Train 50d model with trainable embeds

In [14]:
#LSTM, average final
model50_trained = nn.create_model(embedding_matrix=embedding_matrix50, n_hidden=150, train_embed=True)
model50_trained.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
joke_seq (InputLayer)        (None, 300)               0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 50)           1369800   
_________________________________________________________________
mask_paddings (Masking)      (None, 300, 50)           0         
_________________________________________________________________
drop_words (SpatialDropout1D (None, 300, 50)           0         
_________________________________________________________________
mask_dropped_words (Masking) (None, 300, 50)           0         
_________________________________________________________________
reccurrent_layer (LSTM)      (None, 150)               120600    
_________________________________________________________________
drop_dense (Dropout)         (None, 150)               0         
__________

In [15]:
%%time
history50_trained = nn.run_model(model=model50_trained, out_path="models/nn_50d_trained.hdf5",**run_args)

Train on 171945 samples, validate on 24564 samples
Epoch 1/1000
171945/171945 [==============================] - 152s 886us/step - loss: 0.6728 - acc: 0.5787 - val_loss: 0.6533 - val_acc: 0.6074
Epoch 2/1000
171945/171945 [==============================] - 95s 550us/step - loss: 0.6614 - acc: 0.6023 - val_loss: 0.6437 - val_acc: 0.6231
Epoch 3/1000
171945/171945 [==============================] - 95s 550us/step - loss: 0.6555 - acc: 0.6106 - val_loss: 0.6425 - val_acc: 0.6285
Epoch 4/1000
171945/171945 [==============================] - 94s 549us/step - loss: 0.6521 - acc: 0.6167 - val_loss: 0.6384 - val_acc: 0.6377
Epoch 5/1000
171945/171945 [==============================] - 94s 549us/step - loss: 0.6494 - acc: 0.6213 - val_loss: 0.6344 - val_acc: 0.6402
Epoch 6/1000
171945/171945 [==============================] - 95s 550us/step - loss: 0.6475 - acc: 0.6252 - val_loss: 0.6314 - val_acc: 0.6470
Epoch 7/1000
171945/171945 [==============================] - 95s 550us/step - loss: 0.647

Epoch 28/1000
171945/171945 [==============================] - 94s 549us/step - loss: 0.6206 - acc: 0.6600 - val_loss: 0.6227 - val_acc: 0.6612
Epoch 29/1000
171945/171945 [==============================] - 94s 547us/step - loss: 0.6199 - acc: 0.6596 - val_loss: 0.6233 - val_acc: 0.6622
Epoch 30/1000
171945/171945 [==============================] - 94s 549us/step - loss: 0.6193 - acc: 0.6613 - val_loss: 0.6224 - val_acc: 0.6639
Epoch 31/1000
171945/171945 [==============================] - 94s 548us/step - loss: 0.6185 - acc: 0.6622 - val_loss: 0.6237 - val_acc: 0.6647
Epoch 32/1000
171945/171945 [==============================] - 94s 549us/step - loss: 0.6176 - acc: 0.6638 - val_loss: 0.6208 - val_acc: 0.6645
Epoch 33/1000
171945/171945 [==============================] - 94s 548us/step - loss: 0.6164 - acc: 0.6646 - val_loss: 0.6212 - val_acc: 0.6655
Epoch 34/1000
171945/171945 [==============================] - 94s 549us/step - loss: 0.6156 - acc: 0.6657 - val_loss: 0.6236 - val_acc:

Epoch 58/1000
171945/171945 [==============================] - 94s 549us/step - loss: 0.6017 - acc: 0.6799 - val_loss: 0.6224 - val_acc: 0.6701
Epoch 59/1000
171945/171945 [==============================] - 94s 548us/step - loss: 0.6007 - acc: 0.6813 - val_loss: 0.6209 - val_acc: 0.6710
Epoch 60/1000
171945/171945 [==============================] - 94s 548us/step - loss: 0.6004 - acc: 0.6814 - val_loss: 0.6230 - val_acc: 0.6700
Epoch 61/1000
171945/171945 [==============================] - 94s 549us/step - loss: 0.5992 - acc: 0.6830 - val_loss: 0.6246 - val_acc: 0.6707
Epoch 62/1000
171945/171945 [==============================] - 94s 549us/step - loss: 0.5994 - acc: 0.6831 - val_loss: 0.6230 - val_acc: 0.6707
Epoch 63/1000
171945/171945 [==============================] - 94s 548us/step - loss: 0.5988 - acc: 0.6825 - val_loss: 0.6248 - val_acc: 0.6702
Epoch 64/1000
171945/171945 [==============================] - 94s 549us/step - loss: 0.5983 - acc: 0.6841 - val_loss: 0.6216 - val_acc:

# 5 - Train 300d model with trainable embeds

In [16]:
#LSTM, average final
model300_trained = nn.create_model(embedding_matrix=embedding_matrix300, n_hidden=150, train_embed=True)
model300_trained.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
joke_seq (InputLayer)        (None, 300)               0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 300)          8218800   
_________________________________________________________________
mask_paddings (Masking)      (None, 300, 300)          0         
_________________________________________________________________
drop_words (SpatialDropout1D (None, 300, 300)          0         
_________________________________________________________________
mask_dropped_words (Masking) (None, 300, 300)          0         
_________________________________________________________________
reccurrent_layer (LSTM)      (None, 150)               270600    
_________________________________________________________________
drop_dense (Dropout)         (None, 150)               0         
__________

In [17]:
%%time
history300_trained = nn.run_model(model=model300_trained, out_path="models/nn_300d_trained.hdf5", **run_args)

Train on 171945 samples, validate on 24564 samples
Epoch 1/1000
171945/171945 [==============================] - 173s 1ms/step - loss: 0.6626 - acc: 0.5975 - val_loss: 0.6424 - val_acc: 0.6341
Epoch 2/1000
171945/171945 [==============================] - 108s 627us/step - loss: 0.6434 - acc: 0.6317 - val_loss: 0.6314 - val_acc: 0.6519
Epoch 3/1000
171945/171945 [==============================] - 108s 626us/step - loss: 0.6298 - acc: 0.6503 - val_loss: 0.6262 - val_acc: 0.6617
Epoch 4/1000
171945/171945 [==============================] - 108s 627us/step - loss: 0.6235 - acc: 0.6582 - val_loss: 0.6222 - val_acc: 0.6634
Epoch 5/1000
171945/171945 [==============================] - 108s 627us/step - loss: 0.6167 - acc: 0.6658 - val_loss: 0.6190 - val_acc: 0.6690
Epoch 6/1000
171945/171945 [==============================] - 107s 625us/step - loss: 0.6122 - acc: 0.6702 - val_loss: 0.6198 - val_acc: 0.6702
Epoch 7/1000
171945/171945 [==============================] - 108s 627us/step - loss: 0

Epoch 30/1000
171945/171945 [==============================] - 108s 626us/step - loss: 0.5518 - acc: 0.7209 - val_loss: 0.6446 - val_acc: 0.6742
Epoch 31/1000
171945/171945 [==============================] - 108s 627us/step - loss: 0.5512 - acc: 0.7207 - val_loss: 0.6472 - val_acc: 0.6751
Epoch 32/1000
171945/171945 [==============================] - 108s 626us/step - loss: 0.5484 - acc: 0.7222 - val_loss: 0.6429 - val_acc: 0.6760
Epoch 33/1000
171945/171945 [==============================] - 108s 626us/step - loss: 0.5467 - acc: 0.7248 - val_loss: 0.6479 - val_acc: 0.6753
Epoch 34/1000
171945/171945 [==============================] - 108s 626us/step - loss: 0.5455 - acc: 0.7257 - val_loss: 0.6509 - val_acc: 0.6756
Epoch 35/1000
171945/171945 [==============================] - 108s 626us/step - loss: 0.5434 - acc: 0.7279 - val_loss: 0.6481 - val_acc: 0.6746
Epoch 00035: early stopping
Wall time: 1h 4min 55s


# 6 -  Analyze Best